In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install sklearn_crfsuite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.0 MB/s eta 0:00:00


In [3]:
#importing all the needed libraries
import pandas as pd
import numpy as np
import sklearn
import sklearn_crfsuite
import scipy.stats
import math, string, re
from sklearn_crfsuite import metrics
from itertools import chain

In [4]:
train_set=pd.read_csv('/content/drive/MyDrive/CoLi-Dravidian_2024/mal_train.csv')
val_set=pd.read_csv('/content/drive/MyDrive/CoLi-Dravidian_2024/mal_val.csv')

In [5]:
train_set.head()

,Word,Tag
0,Supr,ENGLISH
1,trailer,ENGLISH
2,oru,MALAYALAM
3,vettame,MALAYALAM
4,like,ENGLISH


In [6]:
print(len(train_set), len(val_set))

24995 2504


In [7]:
train_set['Tag'].value_counts()

Tag
MALAYALAM    11233
ENGLISH       5492
SYM           2777
OTHER         2108
NAME          1941
MIXED          762
NUMBER         568
PLACE          114
Name: count, dtype: int64

In [8]:
np.savetxt(r'/content/drive/MyDrive/CoLi-Dravidian_2024/maltrain.txt', train_set.values, fmt='%s')

In [9]:
np.savetxt(r'/content/drive/MyDrive/CoLi-Dravidian_2024/malval.txt', val_set.values, fmt='%s')

In [10]:
train_word= []
train_tag= []
read_file = open('/content/drive/MyDrive/CoLi-Dravidian_2024/maltrain.txt', 'r',encoding='utf-8')
for line in read_file:
  line = line.replace('\n','')
  items = line.split( )
  train_word.append(items[0])
  train_tag.append(items[1])
read_file.close()

In [ ]:
train_word

In [12]:
line_train= []
train_words=[]
train_tags=[]
i=1
sp= '\n'
for WORD, TAG in zip(train_word, train_tag):
    if WORD != '.':
        line_train.append(float(i))
        train_words.append(WORD)
        train_tags.append(TAG)
        i += 1
    else:
        line_train.append(float(i))
        train_words.append('.')
        train_tags.append(TAG)
        i=1

In [13]:
len(line_train),len(train_words)

(24995, 24995)

In [ ]:
line_train

In [ ]:
train_tags

In [16]:
val_word= []
val_tag= []
read_file1 = open('/content/drive/MyDrive/CoLi-Dravidian_2024/malval.txt', 'r',encoding='utf-8')
for line in read_file1:
  line = line.replace('\n','')
  items = line.split( )
  val_word.append(items[0])
  val_tag.append(items[1])
read_file1.close()

In [17]:
len(val_word)

2504

In [18]:
line_val = []
val_words=[]
val_tags=[]
i=1
sp= '\n'
for WORD, TAG in zip(val_word, val_tag):
    if WORD != '.':
        line_val.append(float(i))
        val_words.append(WORD)
        val_tags.append(TAG)
        i += 1
    else:
        line_val.append(float(i))
        val_words.append('.')
        val_tags.append(TAG)
        i=1

In [19]:
len(val_words)

2504

In [ ]:
val_tags

In [21]:
# from sklearn.model_selection import train_test_split
data={}
dict = {'ID': line_train,'WORD': train_words,'LABEL': train_tags}
data_f= pd.DataFrame(dict)

data['train'] = data_f

data['train']

,ID,WORD,LABEL
0,1.0,Supr,ENGLISH
1,2.0,trailer,ENGLISH
2,3.0,oru,MALAYALAM
3,4.0,vettame,MALAYALAM
4,5.0,like,ENGLISH
...,...,...,...
24990,5.0,oru,MALAYALAM
24991,6.0,hendhi,OTHER
24992,7.0,movie,MALAYALAM
24993,8.0,alla,MALAYALAM


In [22]:
#function to extract features
def word2features(sent, i):
    word = sent[i][0]
    features = {
        'bias': 1.0,
        'word': word,
        'len(word)': len(word),
        'word[:4]': word[:4],
        'word[:3]': word[:3],
        'word[:2]': word[:2],
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word[-4:]': word[-4:],
        'word.ispunctuation': (word in string.punctuation),
        'word.isdigit()': word.isdigit(),
    }
    if i > 0:
        word1l = sent[i-1][0]
        features.update({
            '-1:word': word1l,
            '-1:len(word)': len(word1l),
            '-1:word[:3]': word1l[:3],
            '-1:word[:2]': word1l[:2],
            '-1:word[-3:]': word1l[-3:],
            '-1:word[-2:]': word1l[-2:],
            '-1:word.isdigit()': word1l.isdigit(),
            '-1:word.ispunctuation': (word1l in string.punctuation),
        })

    else:
        features['BOS'] = True

    if i > 1:
        word2l = sent[i-2][0]
        features.update({
            '-2:word': word2l,
            '-2:len(word)': len(word2l),
            '-2:word[:3]': word2l[:3],
            '-2:word[:2]': word2l[:2],
            '-2:word[-3:]': word2l[-3:],
            '-2:word[-2:]': word2l[-2:],
            '-2:word.isdigit()': word2l.isdigit(),
            '-2:word.ispunctuation': (word2l in string.punctuation),
        })

    if i < len(sent)-1:
        word1r = sent[i+1][0]
        features.update({
            '+1:word': word1r,
            '+1:len(word)': len(word1r),
            '+1:word[:3]': word1r[:3],
            '+1:word[:2]': word1r[:2],
            '+1:word[-3:]': word1r[-3:],
            '+1:word[-2:]': word1r[-2:],
            '+1:word.isdigit()': word1r.isdigit(),
            '+1:word.ispunctuation': (word1r in string.punctuation),
        })

    else:
        features['EOS'] = True

    if i < len(sent) - 2:
        word2r = sent[i+2][0]
        features.update({
            '+2:word': word2r,
            '+2:len(word)': len(word2r),
            '+2:word[:3]': word2r[:3],
            '+2:word[:2]': word2r[:2],
            '+2:word[-3:]': word2r[-3:],
            '+2:word[-2:]': word2r[-2:],
            '+2:word.isdigit()': word2r.isdigit(),
            '+2:word.ispunctuation': (word2r in string.punctuation),
        })

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [word[1] for word in sent]
'''
def sent2tokens(sent):
    return [word[0] for word in sent]
'''

'\ndef sent2tokens(sent):\n    return [word[0] for word in sent]\n'

In [23]:
def format_data(csv_data):
    sents = []
    for i in range(len(csv_data)):

        if math.isnan(csv_data.iloc[i, 0]):
            continue
        elif csv_data.iloc[i, 0] == 1.0:
            sents.append([[csv_data.iloc[i, 1], csv_data.iloc[i, 2]]])

        else:
            sents[-1].append([csv_data.iloc[i, 1], csv_data.iloc[i, 2]])
    for sent in sents:
        for i, word in enumerate(sent):
            if type(word[0]) != str:
                del sent[i]
    return sents

In [24]:
dat={}
dict = {'ID': line_val,'WORD': val_words,'LABEL': val_tags}
dat_f= pd.DataFrame(dict)
dat['test']=dat_f

In [25]:
dat_f.head()

,ID,WORD,LABEL
0,1.0,hoo,MALAYALAM
1,2.0,mammokka,NAME
2,3.0,police,ENGLISH
3,4.0,vesham,MALAYALAM
4,5.0,aaha,MALAYALAM


In [26]:
train_sents = format_data(data['train'])
test_sents = format_data(dat['test'])

Xtrain = [sent2features(s) for s in train_sents]
ytrain = [sent2labels(s) for s in train_sents]

Xtest = [sent2features(s) for s in test_sents]
ytest = [sent2labels(s) for s in test_sents]

In [27]:
crf = sklearn_crfsuite.CRF(
    algorithm = 'lbfgs',
    c1 = 0.065,
    c2 = 0.002,
    max_iterations = 200,
    all_possible_transitions=True
)

In [28]:
try:
    crf.fit(Xtrain, ytrain)
except AttributeError:
    pass
predictions = crf.predict(Xtest)

In [29]:
prediction = crf.predict(Xtest)

In [30]:
print('F1 score on the test set = {}\n'.format(metrics.flat_f1_score(ytest, predictions,average='weighted')))#, labels=labels)))
print('Accuracy on the test set = {}\n'.format(metrics.flat_accuracy_score(ytest, predictions)))

F1 score on the test set = 0.9052760935918633

Accuracy on the test set = 0.9089456869009584

